# Problem Statement

You have to write a python script which can fetch all the tweets(as many as allowed by Twitter
API) done by midas@IIITD twitter handle and dump the responses into JSONlines file.<br>
The other part of your script should be able to parse these JSONline files to display the
following for every tweet in a tabular format.

<ul style="list-style-type:disc;">
  <li>The text of the tweet.</li>
  <li>Date and time of the tweet.</li>
  <li>The number of favorites/likes.</li>
  <li>The number of retweets.</li>
  <li>Number of Images present in Tweet. If no image returns None.</li>
</ul>

# Solution Code

In [14]:
import tweepy           #Tweepy supports accessing Twitter via OAuth.It provides access to the well documented Twitter API.
import csv              #csv module to store the tweets in tabular format(csv)
import sys              #System-specific parameters and functions
import jsonlines        #Python library to simplify working with jsonlines and ndjson data.
import pandas as pd     #Pandas library to read csv

In [15]:
#Authorization Credential
access_token = "1728173515-o0TkxRQYMHpD3nIiYWPbNmAAkqZwbuBt70ZWc9T"
access_token_secret = "2LlmmXLC8165jyCnZpSYspiBwnfUoQu7ElkflRQzZMnKL"
consumer_key = "rh7jM7u6EJhb9u6fUNSXLuKM4"
consumer_secret = "CboDAQESB5wGKGmDYtYQd38Oyaqf9aW25egeBss00mP86t0yKm"

In [16]:
#get_tweets function extract all 200 latest tweets tweeted by midas@IIITD account and dump that data into json files
def get_tweets(screen_name):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    #all_tweets variable stores all the tweets tweeted by this account
    all_tweets = []  
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,tweet_mode='extended')
    all_tweets.extend(new_tweets)
    oldest = all_tweets[-1].id - 1
    while len(new_tweets) > 0:
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        all_tweets.extend(new_tweets)
        oldest = all_tweets[-1].id - 1
        print ("...%s number of tweets downloaded so far" % (len(all_tweets)))
    #dumping tweets in jsonline file named as "tweets.jsonl"
    fp = "tweets.jsonl"
    with jsonlines.open('tweets.jsonl', mode='w') as writer:
        for i in all_tweets:  
             writer.write(i._json)

In [17]:
#convertor function to convert json file to csv file
def convertor(reader):
    out_tweets = []
    for t in reader:
                #skippng the media of those tweets which does not have media url              
               if "extended_entities" in t.keys():
                   pr = len(t['extended_entities']['media'])
               else:
                    pr = "None"
               if "full_text" not in t.keys():
                     break
               out_tweets.append([t['created_at'],t['full_text'].encode('utf'),t['favorite_count'],t['retweet_count'],pr])
            
            
    with open('%s_tweets.csv' % "midas", 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["Post Date and Time","Text","Favorite_count","Retweet_count","Images_count"])
        writer.writerows(out_tweets)
    

In [18]:
if __name__ == '__main__':
    #pass the username 'midasiiitd' whose tweets had to download
    get_tweets("midasiiitd")
    #tweets stored in csv file named midas_tweets.csv
    

...344 number of tweets downloaded so far
...344 number of tweets downloaded so far


In [19]:
#opening the tweets.jsonl file in reader mode and converting into csv file named midas_tweets.csv by calling function 'convertor'
reader = jsonlines.open('tweets.jsonl', mode='r')
#calling convertor function to covert file from json to csv
convertor(reader)       
reader.close()

In [21]:
df = pd.read_csv("midas_tweets.csv",header=0)
df


,Post Date and Time,Text,Favorite_count,Retweet_count,Images_count
0,Wed Apr 10 09:01:29 +0000 2019,b'Clarification: Our earlier post which indica...,0,0,None
1,Wed Apr 10 04:51:26 +0000 2019,b'RT @IIITDelhi: Applications open for MTech (...,0,1,None
2,Tue Apr 09 16:45:07 +0000 2019,b'RT @IIITDelhi: We are delighted to share tha...,0,14,None
3,Tue Apr 09 05:04:27 +0000 2019,b'RT @Harvard: Professor Jelani Nelson founded...,0,36,None
4,Tue Apr 09 05:04:11 +0000 2019,b'RT @emnlp2019: For anyone interested in subm...,0,17,None
5,Mon Apr 08 19:38:09 +0000 2019,b'RT @multimediaeval: Announcing the 2019 Medi...,0,15,None
6,Mon Apr 08 07:08:12 +0000 2019,"b'Many Congratulations to @midasIIITD student,...",18,2,1
7,Mon Apr 08 03:27:42 +0000 2019,b'@midasIIITD thanks all students who have app...,5,0,1
8,Sun Apr 07 14:17:29 +0000 2019,"b'@himanchalchandr Meanwhile, complete CV/NLP ...",0,0,None
9,Sun Apr 07 14:17:09 +0000 2019,b'@sayangdipto123 Submit as per the guideline ...,0,0,None
